# RAG Student Assistant

## Ingestion

In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [2]:
# Load pdf
loader = PyPDFLoader('./test.pdf')
docs = loader.load()

In [ ]:
print(len(docs))
print(docs[0].page_content)

1
Introduction au Machine Learning
Le Machine Learning (apprentissage automatique) est une branche de l'intelligence artificielle qui
permet aux ordinateurs d'apprendre à partir des données sans être explicitement programmés.
Il existe trois grandes catégories : 1) L'apprentissage supervisé, 2) L'apprentissage non supervisé,
3) L'apprentissage par renforcement.
Exemple : Un algorithme peut apprendre à reconnaître les chiffres manuscrits en observant des
milliers d'exemples annotés.



### Chunking

In [9]:
# Apply Recursive Character Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5,
    separators=['\n\n', '\n', ' ', '']
)

chunks = splitter.split_text(docs[0].page_content)

In [10]:
print(chunks)

['Introduction au', 'au Machine Learning', 'Le Machine Learning', '(apprentissage', 'automatique) est', 'est une branche de', "de l'intelligence", 'artificielle qui', 'permet aux', 'aux ordinateurs', "d'apprendre à", 'à partir des', 'des données sans', 'sans être', 'être explicitement', 'programmés.', 'Il existe trois', 'grandes catégories', ': 1)', "1) L'apprentissage", 'supervisé, 2)', "2) L'apprentissage", 'non supervisé,', "3) L'apprentissage", 'par renforcement.', 'Exemple : Un', ': Un algorithme', 'peut apprendre à', 'à reconnaître les', 'les chiffres', 'manuscrits en', 'en observant des', "milliers d'exemples", 'annotés.']
